## Functions

In [3]:
import io
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os, sys
from copy import deepcopy
import time

root = "/u/scratch/k/kzheng/"
fs = "/u/scratch/k/kzheng/fs/"
original = "/u/home/k/kzheng/PNet/pnet_prostate_paper/_logs/p1000/pnet/onsplit_average_reg_10_tanh_large_testing/original/"
rerun = "/u/home/k/kzheng/PNet/pnet_prostate_paper/_logs/p1000/pnet/onsplit_average_reg_10_tanh_large_testing/rerun/"

In [11]:
def readData(layer, num=None):
    if num == None:
        df = pd.read_csv(original + "coef_P-net_ALL_layerh" + str(layer) + ".csv")
        df.rename(columns={'Unnamed: 0': 'gene'}, inplace=True)
    else:
        df = pd.read_csv(rerun + "coef_P-net_" + str(num) + "_layerh" + str(layer) + ".csv")
        df.rename(columns={'Unnamed: 0': 'gene'}, inplace=True)
    df["coef"] = df["coef"].abs()
    return df

def readPermutedData(layer, num):
    df = pd.read_csv(fs + "coef_P-net_" + str(num) + "_layerh" + str(layer) + ".csv")
    df.rename(columns={'Unnamed: 0': 'gene'}, inplace=True)
    df["coef"] = df["coef"].abs()
    return df
    
def setBase(df):
    df.rename(columns={'coef': "original"}, inplace=True)
    
def merge(base, df, count):
    base[count] = df["coef"]
    
def loadPermutation(layer, count, model_num=None):
    base = readData(layer, model_num)
    setBase(base)
    for i in range(0, count):
        df = readPermutedData(layer, i)
        merge(base, df, i)
    return base

def rowOP(row):
    l = row[list(range(100))].to_numpy()
    if row["original"] == 0:
        row["pval"] = 1
    else:
        row["pval"] = sum(score > row["original"] for score in l) / len(l)
    return row
    
# find p-value of each SNP
def p_values(df):
    df["pval"] = 1
    df = df.apply(rowOP, axis=1)
    return df

# get a list of causal SNPs given p-values and thresholds
def top_importance(df, threshold):
    genes = []
    for i in range(len(df)):
        if df.iloc[i]["pval"] <= threshold:
            genes.append((i, df.iloc[i]["gene"]))
    return genes

## Layer 0 

In [27]:
layer0 = loadPermutation(0, 100, 1)
layer0 = p_values(layer0)
l = top_importance(layer0, 0.05)

/tmp/ipykernel_12859/1942234564.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base[count] = df["coef"]
/tmp/ipykernel_12859/1942234564.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base[count] = df["coef"]
/tmp/ipykernel_12859/1942234564.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()

In [23]:
layer0["pval"].unique()

array([0.33, 1.  , 0.69, 0.19, 0.55, 0.15, 0.24, 0.47, 0.03, 0.36, 0.42,
       0.46, 0.04, 0.93, 0.17, 0.3 , 0.67, 0.65, 0.99, 0.51, 0.38, 0.86,
       0.5 , 0.97, 0.6 , 0.76, 0.87, 0.71, 0.06, 0.84, 0.  , 0.02, 0.01,
       0.79, 0.9 , 0.12, 0.32, 0.74, 0.52, 0.58, 0.27, 0.18, 0.78, 0.08,
       0.44, 0.21, 0.39, 0.37, 0.09, 0.68, 0.94, 0.29, 0.11, 0.49, 0.77,
       0.98, 0.64, 0.14, 0.41, 0.62, 0.85, 0.23, 0.83, 0.05, 0.91, 0.45,
       0.35, 0.31, 0.13, 0.72, 0.26, 0.25, 0.07, 0.89, 0.4 , 0.95, 0.8 ,
       0.66, 0.43, 0.88, 0.7 , 0.2 , 0.92, 0.34, 0.81, 0.54, 0.63, 0.53,
       0.28, 0.61, 0.48, 0.73, 0.22, 0.1 , 0.16, 0.59, 0.96, 0.56, 0.57,
       0.75, 0.82])

In [28]:
len(l)

540

In [10]:
layer0

,gene,original,0,1,2,3,4,5,6,7,...,91,92,93,94,95,96,97,98,99,pval
0,RNF14,0.049251,0.001178,0.001123,7.518769e-03,0.042259,1.194319e-02,0.000019,0.011329,0.320394,...,3.389958e-13,0.020245,0.004356,4.012160e-03,9.445685e-02,3.178852e-02,6.659929e-04,4.709646e-03,0.027863,0.33
1,RNF17,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.00
2,RNF10,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.00
3,RNF11,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.00
4,RNF13,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9224,PLEKHG5,0.338027,0.064383,0.064453,5.542754e-02,0.121729,1.414231e-01,0.000089,0.745864,0.000397,...,1.398133e-01,0.011674,0.008048,3.495553e-03,4.504771e-01,1.562086e-01,3.745277e-01,2.728618e-05,0.027624,0.06
9225,TGFBR3,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.00
9226,GNGT1,0.028602,0.111557,0.111405,1.506661e-01,0.004746,3.054365e-01,0.000609,0.054017,0.000142,...,2.072499e-02,0.034201,0.054583,6.344729e-02,7.328641e-02,1.712517e-02,1.537914e-02,4.362420e-01,0.000049,0.57
9227,NFIX,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.00


## Layer 1

In [11]:
layer1 = loadPermutation(1, 100)
layer1 = p_values(layer1)
l = top_importance(layer1, 0.05)

/tmp/ipykernel_4914/2894240959.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base[count] = df["coef"]
/tmp/ipykernel_4914/2894240959.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base[count] = df["coef"]
/tmp/ipykernel_4914/2894240959.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
 

In [12]:
len(l)

157

## Layer 2

In [26]:
layer2 = loadPermutation(2, 100)
layer2 = p_values(layer2)
l = top_importance(layer2, 0.05)

/tmp/ipykernel_12859/1942234564.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base[count] = df["coef"]
/tmp/ipykernel_12859/1942234564.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base[count] = df["coef"]
/tmp/ipykernel_12859/1942234564.py:41: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()

In [14]:
len(l)

112

## Layer 3

In [11]:
layer3 = loadPermutation(3, 100)
layer3 = p_values(layer3)
l = top_importance(layer3, 0.05)

/tmp/ipykernel_14498/2040975357.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base[count] = df["coef"]
/tmp/ipykernel_14498/2040975357.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base[count] = df["coef"]
/tmp/ipykernel_14498/2040975357.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()

In [12]:
len(l)

81

## Layer 4

In [15]:
layer4 = loadPermutation(4, 100)
layer4 = p_values(layer4)
l = top_importance(layer4, 0.05)

/tmp/ipykernel_4914/2894240959.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base[count] = df["coef"]
/tmp/ipykernel_4914/2894240959.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base[count] = df["coef"]
/tmp/ipykernel_4914/2894240959.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
 

In [16]:
len(l)

27

In [17]:
layer4

,gene,original,0,1,2,3,4,5,6,7,...,91,92,93,94,95,96,97,98,99,pval
0,R-HSA-9006936,2.310499e-02,2.279284e-04,2.278466e-04,8.381681e-03,3.987842e-03,4.897469e-04,1.796722e-05,1.138944e-02,4.740604e-06,...,6.035870e-05,4.852362e-02,1.513166e-04,7.085444e-04,1.645121e-03,0.000006,2.849894e-05,3.241477e-03,4.304211e-05,0.10
1,R-HSA-1368071,1.965697e-08,0.000000e+00,0.000000e+00,9.627909e-08,5.662374e-09,5.490224e-08,3.035778e-06,3.616813e-05,1.366621e-06,...,3.757950e-07,6.081877e-07,8.000598e-09,1.613948e-07,4.815179e-04,0.000039,0.000000e+00,6.258026e-07,2.793095e-07,0.57
2,R-HSA-157118,2.826089e+00,5.508351e-01,5.506814e-01,7.251450e-01,3.368174e-03,2.328104e-03,9.057184e-02,1.591363e-02,1.491914e-01,...,3.784634e-03,6.800730e-02,4.372453e-02,1.618339e-01,9.000275e-01,0.037558,4.261814e-03,1.774311e-02,9.457885e-05,0.00
3,R-HSA-71737,0.000000e+00,1.702824e-08,1.702791e-08,0.000000e+00,1.427502e-09,0.000000e+00,0.000000e+00,0.000000e+00,3.633266e-09,...,0.000000e+00,1.335933e-05,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,1.00
4,R-HSA-1592230,2.087509e-10,7.114786e-04,7.111722e-04,1.359462e-04,2.032949e-06,2.361284e-05,9.828565e-06,6.722924e-05,1.121588e-06,...,6.901819e-09,6.940975e-07,1.108264e-04,2.069639e-03,1.094423e-04,0.000543,1.351973e-04,8.822049e-02,9.960236e-06,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,R-HSA-5660526,2.050515e-10,0.000000e+00,0.000000e+00,1.043550e-07,0.000000e+00,0.000000e+00,2.382284e-04,7.733158e-06,2.767316e-04,...,2.701554e-07,0.000000e+00,1.738044e-09,7.745151e-05,5.493369e-05,0.000000,1.065028e-09,2.436040e-08,0.000000e+00,0.52
143,R-HSA-445355,2.735055e-08,1.589929e-04,1.590307e-04,1.509825e-02,1.972291e-03,5.513595e-06,4.307636e-05,1.139469e-04,1.998451e-03,...,0.000000e+00,4.619731e-04,2.347757e-04,1.296741e-09,0.000000e+00,0.003754,4.516231e-08,1.118361e-06,2.263681e-02,0.84
144,R-HSA-5223345,6.431515e-07,4.145328e-05,4.149550e-05,1.730141e-05,4.528956e-02,2.760784e-09,1.215772e-04,3.700311e-04,2.578464e-03,...,0.000000e+00,1.334285e-08,2.299137e-08,3.576582e-05,0.000000e+00,0.000387,1.646022e-11,0.000000e+00,0.000000e+00,0.38
145,R-HSA-5682910,1.341577e-08,3.566903e-05,3.565498e-05,0.000000e+00,0.000000e+00,9.299316e-07,2.804422e-10,1.039423e-09,0.000000e+00,...,0.000000e+00,7.945387e-06,3.252650e-09,8.140404e-11,1.528695e-09,0.000000,2.695745e-09,1.423982e-04,2.336590e-03,0.50


## Layer 5

In [18]:
layer5 = loadPermutation(5, 100)
layer5 = p_values(layer5)
l = top_importance(layer5, 0.05)

/tmp/ipykernel_4914/2894240959.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base[count] = df["coef"]
/tmp/ipykernel_4914/2894240959.py:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  base[count] = df["coef"]
/tmp/ipykernel_4914/2894240959.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
 

In [19]:
len(l)

9

In [20]:
layer5

,gene,original,0,1,2,3,4,5,6,7,...,91,92,93,94,95,96,97,98,99,pval
0,R-HSA-397014,6.863754e-05,2.256323e-02,2.258382e-02,1.065560e-02,1.972356e-03,1.034922e-05,0.000029,0.012818,9.912970e-03,...,3.321698e-03,9.700288e-03,1.749171e-03,6.622843e-03,1.207533e-03,0.018857,1.134246e-02,1.547513e-03,0.022818,0.80
1,R-HSA-168256,5.029035e+01,4.070901e+00,4.058387e+00,3.244412e+00,8.150013e-02,5.488005e+00,9.065948,5.961862,5.326350e+00,...,8.124285e+00,7.704827e+00,2.972904e+00,2.042528e+00,7.609162e+00,10.442201,1.058722e+00,2.698065e+00,0.929391,0.00
2,R-HSA-382551,4.467051e-02,1.417649e-04,1.419296e-04,2.777610e-03,6.300511e-02,2.573889e-04,0.044611,0.000947,5.292779e-04,...,1.072913e-03,5.192969e-02,9.983694e-03,4.817984e-03,7.636636e-05,0.001522,5.376917e-04,1.264546e-04,0.022931,0.12
3,R-HSA-1500931,1.011585e-08,2.176852e-04,2.170146e-04,1.736811e-05,2.161382e-01,2.190553e-03,0.023684,0.300558,3.860723e-04,...,1.988189e-04,1.079368e-01,9.417360e-09,5.017657e-08,5.248510e-06,0.000054,9.935345e-03,7.866646e-02,0.004555,0.99
4,R-HSA-5357801,2.057404e-01,7.667931e-02,7.677799e-02,1.502346e-02,1.427826e-02,7.938133e-03,0.053966,0.004834,5.560344e-04,...,5.580279e-03,1.373849e-05,1.460408e-03,8.258923e-04,1.328546e-01,0.000572,2.081616e-03,1.507159e-01,0.074759,0.12
5,R-HSA-8963743,4.133686e-05,2.309491e-06,2.305793e-06,5.357172e-06,1.760370e-04,4.446390e-04,0.001724,0.000627,4.947410e-04,...,1.002321e-08,1.978265e-07,3.879232e-05,2.687702e-10,8.547633e-10,0.000341,3.385887e-08,6.235623e-04,0.000072,0.33
6,R-HSA-1430728,2.096561e+00,6.320185e-02,6.328271e-02,2.783050e-03,1.663404e-02,5.981026e-02,0.000019,0.034837,2.731228e-03,...,4.029207e-02,7.632456e-01,8.803723e-03,9.053007e-03,2.118916e-02,0.021987,3.369707e-02,9.714866e-02,0.014953,0.00
7,R-HSA-1640170,2.854512e+01,3.218846e+00,3.217432e+00,3.802036e+00,2.589647e+00,1.676910e+00,4.364507,0.162745,9.857768e+00,...,2.009866e-02,8.841577e-01,4.083199e+00,3.892945e-01,2.788985e+00,1.477176,4.266749e+00,4.936364e+00,1.042791,0.00
8,R-HSA-1266738,8.154131e-02,5.380452e-02,5.373215e-02,2.204698e-01,3.771131e-01,2.360288e-01,0.152163,0.288272,1.594553e-01,...,2.207018e-02,5.758730e-02,1.298855e+00,2.903057e-02,6.642436e-02,0.757420,4.739473e-02,2.199742e-03,0.314984,0.65
9,R-HSA-74160,4.335403e+01,1.560098e+01,1.560020e+01,1.509457e+01,3.010567e+00,4.770518e-01,19.233953,0.073593,1.091027e+01,...,1.513876e+00,4.971372e-01,8.992366e+00,1.478435e+00,1.670153e+01,10.576977,2.389405e+00,5.923081e+00,0.577065,0.00
